In [1]:
from utils import plot_ship_trajectory
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# Cell 1: Imports & βασικές ρυθμίσεις

# Path για το CSV σου
CSV_PATH = "data/ais_data_final.csv"  # άλλαξέ το αν χρειάζεται

# Hyperparameters για τα παράθυρα
INPUT_LEN = 20     # πόσα ιστορικά σημεία βάζουμε στο input
OUTPUT_LEN = 5     # πόσα μελλοντικά σημεία προβλέπουμε

# Split by MMSI
TRAIN_FRAC = 0.7
VAL_FRAC   = 0.15
TEST_FRAC  = 0.15

RANDOM_STATE = 42

# Model hyperparameters
INPUT_DIM  = 5   # [Latitude, Longtitude, SOG, COG, speed_mps_track]
OUTPUT_DIM = 2   # [Latitude, Longtitude]
HIDDEN_DIM = 128
NUM_LAYERS = 2
DROPOUT    = 0.1
BATCH_SIZE = 128
LR         = 1e-3
NUM_EPOCHS = 20

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [3]:
# Cell 2A: Split the original CSV into train/val/test CSVs by MMSI and save them

import os
import numpy as np
import pandas as pd

def split_and_save_csv_by_mmsi(
    full_csv_path: str,
    train_csv_path: str,
    val_csv_path: str,
    test_csv_path: str,
    train_frac: float = 0.7,
    val_frac: float = 0.15,
    test_frac: float = 0.15,
    random_state: int = 42,
):
    df = pd.read_csv(full_csv_path)

    # Ensure Timestamp is datetime
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])

    # Basic required columns
    required_cols = [
        "Timestamp", "MMSI", "SOG", "COG",
        "Longtitude", "Latitude"
    ]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        print("WARNING: Missing columns:", missing)

    df = df.dropna(subset=["MMSI", "Timestamp", "Latitude", "Longtitude"])

    # Shuffle MMSIs and split
    rng = np.random.RandomState(random_state)
    unique_mmsi = df["MMSI"].unique()
    rng.shuffle(unique_mmsi)

    n = len(unique_mmsi)
    n_train = int(n * train_frac)
    n_val   = int(n * val_frac)

    mmsi_train = unique_mmsi[:n_train]
    mmsi_val   = unique_mmsi[n_train:n_train + n_val]
    mmsi_test  = unique_mmsi[n_train + n_val:]

    df_train = df[df["MMSI"].isin(mmsi_train)].copy()
    df_val   = df[df["MMSI"].isin(mmsi_val)].copy()
    df_test  = df[df["MMSI"].isin(mmsi_test)].copy()

    # Sort each split nicely
    df_train = df_train.sort_values(["MMSI", "Timestamp"])
    df_val   = df_val.sort_values(["MMSI", "Timestamp"])
    df_test  = df_test.sort_values(["MMSI", "Timestamp"])

    # Create folder if needed
    os.makedirs(os.path.dirname(train_csv_path) or ".", exist_ok=True)

    df_train.to_csv(train_csv_path, index=False)
    df_val.to_csv(val_csv_path,   index=False)
    df_test.to_csv(test_csv_path, index=False)

    print(f"Saved train CSV to: {train_csv_path} (rows: {len(df_train)})")
    print(f"Saved val   CSV to: {val_csv_path} (rows: {len(df_val)})")
    print(f"Saved test  CSV to: {test_csv_path} (rows: {len(df_test)})")
    print(f"# MMSI -> train: {len(mmsi_train)}, val: {len(mmsi_val)}, test: {len(mmsi_test)}")

    # Optionally, also save MMSI lists separately if you want
    np.save("mmsi_train.npy", mmsi_train)
    np.save("mmsi_val.npy",   mmsi_val)
    np.save("mmsi_test.npy",  mmsi_test)
    print("Saved mmsi_train.npy, mmsi_val.npy, mmsi_test.npy")

    return df_train, df_val, df_test, mmsi_train, mmsi_val, mmsi_test


In [4]:
# Choose output file names (you can adjust paths if you want)
TRAIN_CSV = "ais_train.csv"
VAL_CSV   = "ais_val.csv"
TEST_CSV  = "ais_test.csv"

df_train_raw, df_val_raw, df_test_raw, mmsi_train, mmsi_val, mmsi_test = split_and_save_csv_by_mmsi(
    full_csv_path=CSV_PATH,      # your original full AIS CSV
    train_csv_path=TRAIN_CSV,
    val_csv_path=VAL_CSV,
    test_csv_path=TEST_CSV,
    train_frac=TRAIN_FRAC,
    val_frac=VAL_FRAC,
    test_frac=TEST_FRAC,
    random_state=RANDOM_STATE,
)


C:\Users\Nikos\AppData\Local\Temp\ipykernel_18316\1146753099.py:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_csv_path)


Saved train CSV to: ais_train.csv (rows: 191877)
Saved val   CSV to: ais_val.csv (rows: 39665)
Saved test  CSV to: ais_test.csv (rows: 45543)
# MMSI -> train: 156, val: 33, test: 35
Saved mmsi_train.npy, mmsi_val.npy, mmsi_test.npy


In [5]:
# Cell 3: Prepare datasets from already-split train/val/test CSV files

def prepare_datasets_from_split_csvs(
    train_csv: str,
    val_csv: str,
    test_csv: str,
    input_len: int = 20,
    output_len: int = 5,
):
    # Load split CSVs
    df_train = pd.read_csv(train_csv)
    df_val   = pd.read_csv(val_csv)
    df_test  = pd.read_csv(test_csv)

    # Make sure Timestamp is datetime type
    for df in [df_train, df_val, df_test]:
        df["Timestamp"] = pd.to_datetime(df["Timestamp"])

    # Sort them
    df_train = df_train.sort_values(["MMSI", "Segment", "Timestamp"])
    df_val   = df_val.sort_values(["MMSI", "Segment", "Timestamp"])
    df_test  = df_test.sort_values(["MMSI", "Segment", "Timestamp"])

    feature_cols_in  = ["Latitude", "Longtitude", "SOG", "COG", "speed_mps_track"]
    feature_cols_out = ["Latitude", "Longtitude"]
    cols_to_scale = list(set(feature_cols_in + feature_cols_out))

    # ---- Fit scaler on TRAIN ONLY ----
    means = {}
    stds  = {}
    for col in cols_to_scale:
        m = df_train[col].mean()
        s = df_train[col].std()
        if s == 0 or np.isnan(s):
            s = 1.0
        means[col] = float(m)
        stds[col]  = float(s)

    scaler = {"mean": means, "std": stds}

    def normalize_df(df_subset):
        df_norm = df_subset.copy()
        for col in cols_to_scale:
            df_norm[col] = (df_norm[col] - means[col]) / stds[col]
        return df_norm

    df_train_n = normalize_df(df_train)
    df_val_n   = normalize_df(df_val)
    df_test_n  = normalize_df(df_test)

    # ---- Build sliding-window sequences per (MMSI, Segment) ----
    def build_sequences(df_subset_norm):
        X_list, Y_list = [], []
        total_len = input_len + output_len

        for (mmsi, seg_id), seg_df in df_subset_norm.groupby(["MMSI", "Segment"]):
            seg_values_in  = seg_df[feature_cols_in].to_numpy(dtype=np.float32)
            seg_values_out = seg_df[feature_cols_out].to_numpy(dtype=np.float32)

            n_pts = len(seg_df)
            if n_pts < total_len:
                continue

            for start in range(0, n_pts - total_len + 1):
                in_slice  = seg_values_in[start : start + input_len]              # (L, 5)
                out_slice = seg_values_out[start + input_len : start + total_len] # (H, 2)
                X_list.append(in_slice)
                Y_list.append(out_slice)

        if not X_list:
            X_arr = np.empty((0, input_len, len(feature_cols_in)), dtype=np.float32)
            Y_arr = np.empty((0, output_len, len(feature_cols_out)), dtype=np.float32)
        else:
            X_arr = np.stack(X_list, axis=0)
            Y_arr = np.stack(Y_list, axis=0)

        return X_arr, Y_arr

    X_train, Y_train = build_sequences(df_train_n)
    X_val,   Y_val   = build_sequences(df_val_n)
    X_test,  Y_test  = build_sequences(df_test_n)

    print(f"Train samples: X{X_train.shape}, Y{Y_train.shape}")
    print(f"Val   samples: X{X_val.shape},   Y{Y_val.shape}")
    print(f"Test  samples: X{X_test.shape},  Y{Y_test.shape}")

    return (X_train, Y_train), (X_val, Y_val), (X_test, Y_test), scaler


In [6]:
# Use the three CSVs instead of the big original file

(train_X, train_Y), (val_X, val_Y), (test_X, test_Y), scaler = prepare_datasets_from_split_csvs(
    train_csv=TRAIN_CSV,
    val_csv=VAL_CSV,
    test_csv=TEST_CSV,
    input_len=INPUT_LEN,
    output_len=OUTPUT_LEN,
)


Train samples: X(186134, 20, 5), Y(186134, 5, 2)
Val   samples: X(38524, 20, 5),   Y(38524, 5, 2)
Test  samples: X(44267, 20, 5),  Y(44267, 5, 2)


In [7]:
# Cell 4: Dataset & DataLoaders

class TrajectoryDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

train_dataset = TrajectoryDataset(train_X, train_Y)
val_dataset   = TrajectoryDataset(val_X,   val_Y)
test_dataset  = TrajectoryDataset(test_X,  test_Y)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False)

len(train_loader), len(val_loader), len(test_loader)


(1455, 301, 346)

In [8]:
# Cell 5: Ορισμός LSTM Encoder–Decoder

class LSTMEncoderDecoder(nn.Module):
    def __init__(self, input_dim=5, output_dim=2,
                 hidden_dim=128, num_layers=2, dropout=0.1):
        super().__init__()

        # Encoder παίρνει όλο το διάνυσμα εισόδου (5 dims)
        self.encoder = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )

        # 🔧 FIX: o decoder να παίρνει επίσης ΟΛΟ το input_dim (5 dims),
        # όχι μόνο τα 2 (lat, lon)
        self.decoder = nn.LSTM(
            input_dim,   # <-- ήταν output_dim
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )

        # Βγάζουμε μόνο (lat, lon)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        """
        src: (B, L, input_dim)   -- normalized
        tgt: (B, H, output_dim)  -- normalized (lat, lon)
        returns: (B, H, output_dim) -- normalized
        """
        batch_size, H, out_dim = tgt.shape

        # Encode ιστορικό
        _, (h, c) = self.encoder(src)  # h, c: (num_layers, B, hidden_dim)

        # 🔧 FIX: Πρώτο input του decoder = πλήρες τελευταίο διάνυσμα (lat, lon, SOG, COG, speed)
        dec_input = src[:, -1, :]          # (B, input_dim=5)
        dec_input = dec_input.unsqueeze(1) # (B, 1, 5)

        outputs = []
        for t in range(H):
            # Decoder step
            dec_out, (h, c) = self.decoder(dec_input, (h, c))  # (B, 1, hidden_dim)
            step_output = self.fc_out(dec_out)                 # (B, 1, 2) -> lat, lon
            outputs.append(step_output)

            # Teacher forcing: επιλέγω αν θα βάλω το ground truth ή την πρόβλεψη
            if self.training and torch.rand(1).item() < teacher_forcing_ratio:
                next_latlon = tgt[:, t:t+1, :]   # (B, 1, 2) ground truth
            else:
                next_latlon = step_output        # (B, 1, 2) prediction

            # 🔧 FIX: κρατάμε SOG/COG/speed όπως ήταν,
            # και αντικαθιστούμε μόνο τα 2 πρώτα στοιχεία (lat, lon)
            dec_input = dec_input.clone()        # (B, 1, 5)
            dec_input[:, :, :2] = next_latlon    # βάζουμε νέο lat, lon

        outputs = torch.cat(outputs, dim=1)  # (B, H, 2)
        return outputs


In [9]:
# Cell 6: Δημιουργία μοντέλου, loss & optimizer

model = LSTMEncoderDecoder(
    input_dim=INPUT_DIM,
    output_dim=OUTPUT_DIM,
    hidden_dim=HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

print(model)


LSTMEncoderDecoder(
  (encoder): LSTM(5, 128, num_layers=2, batch_first=True, dropout=0.1)
  (decoder): LSTM(5, 128, num_layers=2, batch_first=True, dropout=0.1)
  (fc_out): Linear(in_features=128, out_features=2, bias=True)
)


In [11]:
import os
import torch

# --------------------------
# CONFIG
# --------------------------
NUM_EPOCHS = 10  # run for 10 epochs

# 👉 Change this to whatever path/filename you want:
# e.g. "/content/drive/MyDrive/deep learning/models/lstm_best.pth"
MODEL_PATH = "data/lstm_best_2.pth"

# Create directory if it does not exist
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

# --------------------------
# TRAINING LOOP WITH BEST-MODEL SAVING
# --------------------------
best_val_loss = float("inf")

for epoch in range(NUM_EPOCHS):
    # ----- TRAIN -----
    model.train()
    total_train_loss = 0.0

    for X_batch, Y_batch in train_loader:
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)

        optimizer.zero_grad()
        preds = model(X_batch, Y_batch, teacher_forcing_ratio=0.5)
        loss = criterion(preds, Y_batch)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item() * X_batch.size(0)

    avg_train_loss = total_train_loss / len(train_loader.dataset)

    # ----- VALIDATION -----
    model.eval()
    total_val_loss = 0.0

    with torch.no_grad():
        for X_batch, Y_batch in val_loader:
            X_batch = X_batch.to(device)
            Y_batch = Y_batch.to(device)

            preds = model(X_batch, Y_batch, teacher_forcing_ratio=0.0)
            loss = criterion(preds, Y_batch)
            total_val_loss += loss.item() * X_batch.size(0)

    avg_val_loss = total_val_loss / len(val_loader.dataset)

    # ----- SAVE BEST MODEL -----
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss

        torch.save({
            "model_state_dict": model.state_dict(),
            "scaler": scaler,
            "input_len": INPUT_LEN,
            "output_len": OUTPUT_LEN,
        }, MODEL_PATH)

        print(f"🔥 New best model saved to {MODEL_PATH}! Val MSE = {best_val_loss:.6f}")

    # ----- PRINT EPOCH RESULTS -----
    print(
        f"Epoch {epoch+1}/{NUM_EPOCHS} - "
        f"Train MSE: {avg_train_loss:.6f} - Val MSE: {avg_val_loss:.6f} "
        f"(Best Val: {best_val_loss:.6f})"
    )


🔥 New best model saved to data/lstm_best_2.pth! Val MSE = 0.000150
Epoch 1/10 - Train MSE: 0.010706 - Val MSE: 0.000150 (Best Val: 0.000150)
Epoch 2/10 - Train MSE: 0.000399 - Val MSE: 0.000229 (Best Val: 0.000150)
🔥 New best model saved to data/lstm_best_2.pth! Val MSE = 0.000101
Epoch 3/10 - Train MSE: 0.000358 - Val MSE: 0.000101 (Best Val: 0.000101)
Epoch 4/10 - Train MSE: 0.000318 - Val MSE: 0.000182 (Best Val: 0.000101)
Epoch 5/10 - Train MSE: 0.000288 - Val MSE: 0.000148 (Best Val: 0.000101)
Epoch 6/10 - Train MSE: 0.000228 - Val MSE: 0.000477 (Best Val: 0.000101)
Epoch 7/10 - Train MSE: 0.000230 - Val MSE: 0.000146 (Best Val: 0.000101)
Epoch 8/10 - Train MSE: 0.000221 - Val MSE: 0.000127 (Best Val: 0.000101)
Epoch 9/10 - Train MSE: 0.000181 - Val MSE: 0.000310 (Best Val: 0.000101)
Epoch 10/10 - Train MSE: 0.000188 - Val MSE: 0.000143 (Best Val: 0.000101)


In [10]:
# Load best model from file
checkpoint = torch.load(
    "data/lstm_best_2.pth",
    map_location=device,
    weights_only=False,  # <-- add this
)

loaded_model = LSTMEncoderDecoder(
    input_dim=INPUT_DIM,
    output_dim=OUTPUT_DIM,
    hidden_dim=HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
).to(device)

loaded_model.load_state_dict(checkpoint["model_state_dict"])
loaded_model.eval()

loaded_scaler = checkpoint["scaler"]
loaded_input_len  = checkpoint["input_len"]
loaded_output_len = checkpoint["output_len"]

print("Model loaded successfully!")


Model loaded successfully!


In [11]:
def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# --------- EVALUATE LOADED MODEL ---------
loaded_model.eval()

lat_mean = loaded_scaler["mean"]["Latitude"]
lat_std  = loaded_scaler["std"]["Latitude"]
lon_mean = loaded_scaler["mean"]["Longtitude"]
lon_std  = loaded_scaler["std"]["Longtitude"]

all_distances = []
all_mse = 0.0
n_samples = 0

with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)

        preds = loaded_model(X_batch, Y_batch, teacher_forcing_ratio=0.0)

        # MSE in normalized space
        loss = criterion(preds, Y_batch)
        all_mse += loss.item() * X_batch.size(0)
        n_samples += X_batch.size(0)

        preds_np = preds.cpu().numpy()
        true_np  = Y_batch.cpu().numpy()

        # Denormalize
        true_lat = true_np[..., 0] * lat_std + lat_mean
        true_lon = true_np[..., 1] * lon_std + lon_mean
        pred_lat = preds_np[..., 0] * lat_std + lat_mean
        pred_lon = preds_np[..., 1] * lon_std + lon_mean

        # Compute Haversine distance
        d = haversine_m(true_lat, true_lon, pred_lat, pred_lon)
        all_distances.append(d)

all_distances = np.concatenate(all_distances, axis=0)
mean_per_step = all_distances.mean(axis=0)
overall_mean  = all_distances.mean()

avg_test_mse = all_mse / n_samples

print(f"Test MSE (normalized): {avg_test_mse:.6f}")
print("Mean Haversine distance per prediction step (meters):", mean_per_step)
print("Overall mean Haversine distance (meters):", overall_mean)


Test MSE (normalized): 0.000105
Mean Haversine distance per prediction step (meters): [1449.128  1531.5199 1601.2687 1708.5358 1866.7987]
Overall mean Haversine distance (meters): 1631.4495


In [16]:
import numpy as np

FEATURES = ["Latitude", "Longtitude", "SOG", "COG", "speed_mps_track"]


def inverse_transform_array(X, scaler):
    X = np.asarray(X)
    mean = np.array([scaler["mean"][f] for f in FEATURES], dtype=float)
    std  = np.array([scaler["std"][f]  for f in FEATURES], dtype=float)
    return X * std + mean


In [17]:
import folium
import itertools
import torch

FEATURES = ["Latitude", "Longtitude", "SOG", "COG", "speed_mps_track"]
LAT_IDX = FEATURES.index("Latitude")
LON_IDX = FEATURES.index("Longtitude")


def plot_predicted_vs_true_folium(
    model,
    test_loader,
    device,
    scaler,
    batch_idx=0,
    sample_idx=0,
    save_path=None,
):
    """
    PLOTS on Folium:
      - History (blue)
      - True future (green)
      - Predicted future (red)

    Assumes X and Y in test_loader are in NORMALIZED space.
    Uses `scaler` (object or dict) to denormalize back to real degrees.
    """

    model.eval()

    # -------- 1. Get one batch from test_loader --------
    batch = next(itertools.islice(test_loader, batch_idx, batch_idx + 1))
    X_batch, Y_batch = batch  # X: (B, L, F), Y: (B, H, 2 or F)

    X_batch = X_batch.to(device)
    Y_batch = Y_batch.to(device)

    with torch.no_grad():
        preds = model(X_batch, Y_batch, teacher_forcing_ratio=0.0)

    # choose one sample
    X_sample = X_batch[sample_idx].cpu().numpy()       # (L, F) normalized
    Y_true_sample = Y_batch[sample_idx].cpu().numpy()  # (H, 2 or F)
    Y_pred_sample = preds[sample_idx].cpu().numpy()    # (H, 2 or F)

    # -------- 2. Denormalize lat/lon --------
    if Y_true_sample.shape[1] == X_sample.shape[1]:
        # Y has same F features as X (all normalized)
        hist_denorm = inverse_transform_array(X_sample, scaler)
        true_denorm = inverse_transform_array(Y_true_sample, scaler)
        pred_denorm = inverse_transform_array(Y_pred_sample, scaler)

        hist_lat = hist_denorm[:, LAT_IDX]
        hist_lon = hist_denorm[:, LON_IDX]
        true_lat = true_denorm[:, LAT_IDX]
        true_lon = true_denorm[:, LON_IDX]
        pred_lat = pred_denorm[:, LAT_IDX]
        pred_lon = pred_denorm[:, LON_IDX]

    elif Y_true_sample.shape[1] == 2:
        # Denormalize history
        hist_denorm = inverse_transform_array(X_sample, scaler)
        hist_lat = hist_denorm[:, LAT_IDX]
        hist_lon = hist_denorm[:, LON_IDX]

        # Denormalize future ground truth
        true_stack = np.zeros((Y_true_sample.shape[0], len(FEATURES)))
        true_stack[:, :2] = Y_true_sample  # put normalized lat/lon in first 2 columns
        true_denorm = inverse_transform_array(true_stack, scaler)
        true_lat = true_denorm[:, LAT_IDX]
        true_lon = true_denorm[:, LON_IDX]

        # Denormalize future predictions (same trick)
        pred_stack = np.zeros((Y_pred_sample.shape[0], len(FEATURES)))
        pred_stack[:, :2] = Y_pred_sample
        pred_denorm = inverse_transform_array(pred_stack, scaler)
        pred_lat = pred_denorm[:, LAT_IDX]
        pred_lon = pred_denorm[:, LON_IDX]

    else:
        raise ValueError(
            f"Unexpected Y shape {Y_true_sample.shape}; "
            "update the denormalization logic accordingly."
        )

    # -------- 3. Center of map --------
    all_lats = np.concatenate([hist_lat, true_lat, pred_lat])
    all_lons = np.concatenate([hist_lon, true_lon, pred_lon])

    center_lat = float(all_lats.mean())
    center_lon = float(all_lons.mean())

    m = folium.Map(location=[center_lat, center_lon], zoom_start=7)

    # -------- 4. Plot history (blue) --------
    folium.PolyLine(
        list(zip(hist_lat, hist_lon)),
        popup="History",
        tooltip="History",
        weight=3,
    ).add_to(m)

    # -------- 5. Plot true future (green) --------
    folium.PolyLine(
        list(zip(true_lat, true_lon)),
        popup="True future",
        tooltip="True future",
        weight=3,
    ).add_to(m)

    # -------- 6. Plot predicted future (red) --------
    folium.PolyLine(
        list(zip(pred_lat, pred_lon)),
        popup="Predicted future",
        tooltip="Predicted future",
        weight=3,
    ).add_to(m)

    if save_path is not None:
        m.save(save_path)
        print(f"Map saved to {save_path}")

    return m


In [18]:
m = plot_predicted_vs_true_folium(
    model=loaded_model,
    test_loader=test_loader,
    device=device,
    scaler=loaded_scaler,
    batch_idx=0,
    sample_idx=0,
    save_path="pred_vs_true_2.html",
)


Map saved to pred_vs_true_2.html
